In [1]:
import pandas as pd
import statsmodels.api as sm

# 1.) Import Data from FRED

In [2]:
data = pd.read_csv("TaylorRuleData.csv", index_col = 0)

In [3]:
data.index = pd.to_datetime(data.index)

In [7]:
data.head()

,FedFunds,Unemployment,HousingStarts,Inflation
1947-01-01,NaN,NaN,NaN,21.48
1947-02-01,NaN,NaN,NaN,21.62
1947-03-01,NaN,NaN,NaN,22.00
1947-04-01,NaN,NaN,NaN,22.00
1947-05-01,NaN,NaN,NaN,21.95


In [8]:
data.describe()

,FedFunds,Unemployment,HousingStarts,Inflation
count,834.000000,912.000000,779.000000,923.000000
mean,4.601667,5.704934,1433.207959,118.818295
std,3.592438,1.710877,382.765522,84.653938
min,0.050000,2.500000,478.000000,21.480000
25%,1.802500,4.400000,1204.000000,32.230000
50%,4.165000,5.500000,1457.000000,107.500000
75%,6.240000,6.700000,1649.500000,189.500000
max,19.100000,14.900000,2494.000000,307.917000


# 2.) Do Not Randomize, split your data into Train, Test Holdout

In [ ]:
split_1 = 
split_2 = 
data_in = 
data_out = 
data_hold = 

In [ ]:
X_in = 
y_in = 
X_out = 
y_out = 
X_hold = 
y_hold = 

In [ ]:
# Add Constants
X_in = 
X_out =  
X_hold =  

# 3.) Build a model that regresses FF~Unemp, HousingStarts, Inflation

In [ ]:
model1 = 

# 4.) Recreate the graph fro your model

In [ ]:
import matplotlib.pyplot as plt

In [ ]:
plt.figure(figsize = (12,5))

###

###

plt.ylabel("Fed Funds")
plt.xlabel("Time")
plt.title("Visualizing Model Accuracy")
plt.legend([])
plt.grid()
plt.show()

## "All Models are wrong but some are useful" - 1976 George Box

# 5.) What are the in/out of sample MSEs

In [ ]:
from sklearn.metrics import mean_squared_error

In [ ]:
in_mse_1 = 
out_mse_1 = 

In [ ]:
print("Insample MSE : ", in_mse_1)
print("Outsample MSE : ", out_mse_1)

# 6.) Using a for loop. Repeat 3,4,5 for polynomial degrees 1,2,3

In [ ]:
from sklearn.preprocessing import PolynomialFeatures

# 7.) State your observations :